In [1]:
DATAPATH = "data path"
movieNameList = []
f = open(DATAPATH,"r",encoding="ISO-8859-1")
while 1:
    line = f.readline()
    if not line:
        break
    
    #clean data
    line = line.split("::")
    line = line[1].replace("(","@").split("@")[0]
    
    movieNameList.append(line)
        

    

In [2]:
import requests 
from pyquery import PyQuery as pq
import time
import shutil as sh
import os


num=2142
SAVE_PATH = "D:\\IMDB\\"
#check path exist or not
if not os.path.isdir(SAVE_PATH):
    os.mkdir(SAVE_PATH)
    
    
NoneList = []
NoCo = []

for movie in movieNameList[num:]:

    mode = 0
    print(num)
    num+=1
    rawMovieName = movie
    rawMovieName = rawMovieName.replace(":"," ").replace("/"," ").replace("\/"," ").replace("\\"," ").replace("?"," ").replace("*"," ")
    #love and peace :)
    time.sleep(0.001)

    # url query format
    movieName = rawMovieName.replace(" ","+")
    # join with query url

    #print("Searching:",rawMovieName)
    URL = "https://www.imdb.com/"
    queryURL = URL+"find?q="+movieName

    qeuryRes = requests.get(queryURL)

    queryDoc = pq(qeuryRes .text)


    result_url =queryDoc("#main > div > div:nth-child(3) > table > tr > td > a ").attr("href")
    if result_url == None:
        result_url =queryDoc("#main > div > div:nth-child(4)>table>tr>td>a").attr("href")
    if result_url==None or result_url[:6]!="/title":
        result_url= queryDoc("#main > div > div:nth-child(3) > table > tr > td > a ").attr("href")
    #Aimée & Jaguar 

    if result_url==None or result_url[:6]!="/title":
        print("not found  ", result_url,movie)
        NoneList.append(rawMovieName)
        print(NoneList)
        continue
    resultRes = requests.get(URL+result_url)

    resultDoc = pq(resultRes.text)

    #print("checking path exist or not")
    curSavePath= SAVE_PATH+rawMovieName+"\\"

    
 
        


    f = open(curSavePath+"movie_name","w",encoding="ISO-8859-1", errors='ignore')
    f.write(movie)
    f.close()


    print("saving picture")
    #toy story
    imgURL=resultDoc("#title-overview-widget > div.vital > div.slate_wrapper > div.poster > a > img").attr("src")
  
    #Across the Sea of Time 
    if imgURL== None: 
   
        imgURL=resultDoc("#title-overview-widget > div.minPosterWithPlotSummaryHeight > div.poster > a > img").attr("src")
        mode=1
    if imgURL == None:
        NoneList.append(rawMovieName)
        print(NoneList)
        continue
    f=open(curSavePath+"imgurl",'w')
    print(imgURL)
    f.write(imgURL)
    f.close()
    
    if imgURL == None:
        NoneList.append(rawMovieName)
        print(NoneList)
        continue

    #print("saving production_co")
    for i in range(20):
        productions = resultDoc("#titleDetails > div:nth-child("+str(i)+")")
        if productions.text()[:14]=="Production Co:":
            break

   

    productionList=""
    i=2
    while 1 :

        production=productions("a:nth-child("+str(i)+")")
        
        production=production.text()
      

        if production=="" or production==None :
            break
        productionList+=production+"\n"


        i+=1

    if productionList=="":
        NoCo.append(movie)
        print(NoCo)

    #print(movie,productionList)
    f=open(curSavePath+"production_co",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(productionList)
    f.close()
    


    img = requests.get(imgURL,stream=True)
    f=open(curSavePath+"img.jpg",'wb')
    sh.copyfileobj(img.raw,f)
    f.close()


    print("saving intro.")
    text = resultDoc("#title-overview-widget > div.plot_summary_wrapper > div.plot_summary > div.summary_text").text()
    if mode == 1:
        text = resultDoc('#title-overview-widget > div.minPosterWithPlotSummaryHeight > div:nth-child(3) > div.plot_summary > div.summary_text').text()
    if text =="":
        print("warning for ",rawMovieName,"intro")

    f=open(curSavePath+"summary_text",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(text)
    f.close()



    print("saving ratings")
    rate = resultDoc("#title-overview-widget > div.vital > div.title_block > div > div.ratings_wrapper > div.imdbRating > div.ratingValue > strong > span").text()

    if rate =="":
        print("warning for ",rawMovieName,"rating")

    f=open(curSavePath+"rating",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(rate)
    f.close()



    print("saving directors")
    director = resultDoc("#title-overview-widget > div.plot_summary_wrapper > div.plot_summary > div:nth-child(2) > a").text()
    if mode ==1:
         director= resultDoc("#title-overview-widget > div.minPosterWithPlotSummaryHeight > div:nth-child(3) > div.plot_summary > div:nth-child(2) > a").text()
    if director=="":
        print("warning for ",rawMovieName,"director")

    f=open(curSavePath+"director",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(director)
    f.close()



    print("saving writers")
    writers = resultDoc("#title-overview-widget > div.plot_summary_wrapper > div.plot_summary > div:nth-child(3)")
    if mode ==1:
         writers = resultDoc("#title-overview-widget > div.minPosterWithPlotSummaryHeight > div:nth-child(3) > div.plot_summary > div:nth-child(3)")
    writerList=""
    i=2
    while 1 :

        writer=writers("a:nth-child("+str(i)+")")

        writer=writer.text()


        if writer=="" or writer==None :
            break
        writerList+=writer+"\n"


        i+=1

    if writerList==[]:
        print("warning for ",rawMovieName,"writer")

    f=open(curSavePath+"writer",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(writerList)
    f.close()


    print("saving stars")
    stars = resultDoc("#title-overview-widget > div.plot_summary_wrapper > div.plot_summary > div:nth-child(4) ")
    if mode ==1:
        stars = resultDoc("#title-overview-widget > div.minPosterWithPlotSummaryHeight > div:nth-child(3) > div.plot_summary > div:nth-child(4) ")
    starList = ""
    i=2
    while 1 :
        star=stars("a:nth-child("+str(i)+")")
        star=star.text()


        if star=="" or star==None :
            break
        starList+=star+"\n"
        i+=1


    if starList==[]:
        print("warning for ",rawMovieName,"stars")

    f=open(curSavePath+"star",'w',encoding="ISO-8859-1", errors='ignore')
    f.write(starList)
    f.close()


    print("saving storyline")
    storyline = resultDoc("#titleStoryLine > div:nth-child(3) > p > span").text()
    f=open(curSavePath+"storyline",'w',encoding="ISO-8859-1",errors='ignore')
    if storyline=="":
        print("warning for ",rawMovieName,"storyline")
    f.write(storyline)
    f.close()

    """
    #print("finish")
    
    








2142
saving picture
https://m.media-amazon.com/images/M/MV5BMjA4NDk2MzYxM15BMl5BanBnXkFtZTgwNzA4MDYyMDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2143
saving picture
https://m.media-amazon.com/images/M/MV5BNjQ2OWFhMjItYmFmNC00OGQzLWE1ZTMtZGQwZDM3YWUzMDgxL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyMDI2NDg0NQ@@._V1_UX182_CR0,0,182,268_AL_.jpg
2144
saving picture
https://m.media-amazon.com/images/M/MV5BZmJkZWM3MzQtN2ViMS00OWZkLTk0NWItYTg5ZDEwM2QzMzg2XkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UY268_CR1,0,182,268_AL_.jpg
2145
saving picture
https://m.media-amazon.com/images/M/MV5BM2YzYWExOTMtZDRkNi00MTdmLTgyOTQtZmY2ZTA5NWQ0YzU5XkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UY268_CR1,0,182,268_AL_.jpg
2146
saving picture
https://m.media-amazon.com/images/M/MV5BNmVhOTgyMDMtZWNlMC00OWIzLWEzYzctYTNmM2I5MjNlNmFmXkEyXkFqcGdeQXVyNzU3ODA2Mg@@._V1_UY268_CR4,0,182,268_AL_.jpg
2147
saving picture
https://m.media-amazon.com/images/M/MV5BNDgyZTZkNTctMTg1OS00NWJjLTljY2MtZjUzMWM5NGI4YmI5XkEyXkFqcGdeQXVyNjI2ODA4MDU@._V1_UX182_CR0,0,182,268_AL_.jpg
21

saving picture
https://m.media-amazon.com/images/M/MV5BODY3ZjRmODYtYjM0My00ZmRlLWJiZDQtZGQ2Y2NmMDVhMThhXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2194
saving picture
https://m.media-amazon.com/images/M/MV5BNWI0M2E5ZGYtODU2MC00OWUxLWIxMDctMmQxMjZjZjhlMTRmXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2195
saving picture
https://m.media-amazon.com/images/M/MV5BMWQ3YWQ5NjMtNTVlZi00ZTk2LWFjZDctYjNiNzVhYjllMjFiXkEyXkFqcGdeQXVyMzk1NzcyOTA@._V1_UY268_CR0,0,182,268_AL_.jpg
2196
saving picture
https://m.media-amazon.com/images/M/MV5BYjc5MjA1ZWYtNTgxMi00ZTlkLWExZTctMTFkNDFlOTdlZmRkXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2197
saving picture
https://m.media-amazon.com/images/M/MV5BNDk0MWYxOTAtNGYwOC00OWMzLWFhNGEtYWNhNWNiMmEzMTIzXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2198
saving picture
https://m.media-amazon.com/images/M/MV5BMjljOGVlZDMtMGE5ZS00OTIxLWE4ZTYtNjQyNTgyMzk0YTZhXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268

saving picture
https://m.media-amazon.com/images/M/MV5BN2VjN2ViNTItMWFhNS00MTdjLTk0MTEtYTA5MzQ5OThkMjgwXkEyXkFqcGdeQXVyMTA0MjU0Ng@@._V1_UX182_CR0,0,182,268_AL_.jpg
2244
saving picture
https://m.media-amazon.com/images/M/MV5BMDVjNjIwOGItNDE3Ny00OThjLWE0NzQtZTU3YjMzZTZjMzhkXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2245
saving picture
https://m.media-amazon.com/images/M/MV5BYTZlMDFkYjItMjgxYi00MDNhLTg0NDYtNDY1NzJkNDg0MGQ4XkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_UX182_CR0,0,182,268_AL_.jpg
2246
saving picture
https://m.media-amazon.com/images/M/MV5BZTcxZDE5Y2MtODNmOC00YjQ0LTk3YzQtZWU1OTU2ZTgwNDVkXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2247
saving picture
https://m.media-amazon.com/images/M/MV5BMzU4MmNkNGYtN2I1Zi00ZDEzLTlmZjItNWQ5NWI0NzcwZDQxXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2248
saving picture
https://m.media-amazon.com/images/M/MV5BMWQ2ZDdjNWYtNTY1Yy00MGIzLTg4NDQtYTFmYTJjZTFhYzNjXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY268_CR0,0,182,268

saving picture
https://m.media-amazon.com/images/M/MV5BN2E4ZDgxN2YtZjExMS00MWE5LTg3NjQtNTkxMzJhOTA3MDQ4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2295
saving picture
https://m.media-amazon.com/images/M/MV5BN2E4ZDgxN2YtZjExMS00MWE5LTg3NjQtNTkxMzJhOTA3MDQ4XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2296
saving picture
https://m.media-amazon.com/images/M/MV5BYmY0YjBhNTEtZmEyNS00YjllLTkyYTktYzZlYjcyNmQzYjI1L2ltYWdlXkEyXkFqcGdeQXVyNjQ2MzU1NzQ@._V1_UY268_CR3,0,182,268_AL_.jpg
2297
saving picture
https://m.media-amazon.com/images/M/MV5BMjYxYmRlZWYtMzAwNC00MDA1LWJjNTItOTBjMzlhNGMzYzk3XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2298
saving picture
https://m.media-amazon.com/images/M/MV5BMjYxYmRlZWYtMzAwNC00MDA1LWJjNTItOTBjMzlhNGMzYzk3XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
2299
saving picture
https://m.media-amazon.com/images/M/MV5BNzAxNTQyMzYtMTg5Zi00OTA1LWEwYjEtNjg3ZmJhOWZiMzk3XkEyXkFqcGdeQXVyMTQ2MjQyNDc@._V1_UY268_CR1,0

saving picture
https://m.media-amazon.com/images/M/MV5BMTM2Mzc4MTYxM15BMl5BanBnXkFtZTcwODY5NTk1NA@@._V1_UX182_CR0,0,182,268_AL_.jpg
2344
saving picture
https://m.media-amazon.com/images/M/MV5BM2VlNTE1ZmMtOTAyNS00ODYwLWFmY2MtZWEzOTE2YjE1NDE2XkEyXkFqcGdeQXVyNDk3NzU2MTQ@._V1_UX182_CR0,0,182,268_AL_.jpg
2345
saving picture
https://m.media-amazon.com/images/M/MV5BNmE0NTQ5ZjItN2MzNC00NjBhLTg0ZjYtYzFiMzFlMjhjNmNiXkEyXkFqcGdeQXVyNDk3NzU2MTQ@._V1_UX182_CR0,0,182,268_AL_.jpg
2346
saving picture
https://m.media-amazon.com/images/M/MV5BYzljNjY1ZDctZmRjNC00N2QzLWE4ZjQtMzM5MDg2NGQzMDVkXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_UX182_CR0,0,182,268_AL_.jpg
2347
saving picture
https://m.media-amazon.com/images/M/MV5BMTYxMTIyOTM1MF5BMl5BanBnXkFtZTcwNzQ0MTcyNA@@._V1_UX182_CR0,0,182,268_AL_.jpg
2348
saving picture
https://m.media-amazon.com/images/M/MV5BY2I3ODU4MWItZDJmOS00NGFkLTkzYWMtMWI4NTFmNDI2MjNmXkEyXkFqcGdeQXVyMTA0MjU0Ng@@._V1_UX182_CR0,0,182,268_AL_.jpg
2349
saving picture
https://m.media-amazon.com/images/M

...

saving picture
https://m.media-amazon.com/images/M/MV5BNGE3MTQ3YmItYzlkZi00MzIxLWFiZDUtMDA1Y2QwN2Q2YjRmXkEyXkFqcGdeQXVyMzg1ODEwNQ@@._V1_UX182_CR0,0,182,268_AL_.jpg
3665
saving picture
https://m.media-amazon.com/images/M/MV5BNTM5YzVlNWUtNzMxYy00ZGM1LWE3NWQtZDQxNjBlODJkNGU3XkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX182_CR0,0,182,268_AL_.jpg
3666
saving picture
https://m.media-amazon.com/images/M/MV5BYTU4ZTI0NzAtYzMwNi00YmMxLThmZWItNTY5NzgyMDAwYWVhXkEyXkFqcGdeQXVyNjUwNzk3NDc@._V1_UX182_CR0,0,182,268_AL_.jpg
3667
saving picture
https://m.media-amazon.com/images/M/MV5BNDc4NjMxMDgtOWNmNi00ZjE4LTgxZGItMDBmNmJjZjY5ZmFjL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyMTUzMTk2MTc@._V1_UX182_CR0,0,182,268_AL_.jpg
3668
saving picture
https://m.media-amazon.com/images/M/MV5BMTViZDU5MDktNjRjYS00Zjk1LThkNmUtMzgwYmRjMmVkZjU3XkEyXkFqcGdeQXVyNTEwNDMwNA@@._V1_UX182_CR0,0,182,268_AL_.jpg
3669
saving picture
https://m.media-amazon.com/images/M/MV5BM2ZlOTJmYzUtMWQ1ZC00ZDQzLWFjNDUtMDliMjJiMWI5YjBmXkEyXkFqcGdeQXVyMjI4MjA5MzA@._V1_UX1

saving picture
https://m.media-amazon.com/images/M/MV5BYjFjMTg4ZGQtNzgxOS00MThkLWE3YTUtMGMwZDE1ZGYxNmQ0XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UX182_CR0,0,182,268_AL_.jpg
3715
saving picture
https://m.media-amazon.com/images/M/MV5BNTk0ZmRiNGMtNDgwMi00ZTBjLWE4NmYtMzVlNmQ5NTM4NTUxXkEyXkFqcGdeQXVyNjc0MjcxODM@._V1_UX182_CR0,0,182,268_AL_.jpg
3716
saving picture
https://m.media-amazon.com/images/M/MV5BMjA2NjU5MTg5OF5BMl5BanBnXkFtZTgwOTkyMzQxMDE@._V1_UX182_CR0,0,182,268_AL_.jpg
3717
saving picture
https://m.media-amazon.com/images/M/MV5BNmZjNGVmYmItZWFmNi00ODQ1LThmZTUtMzYzMGJlMjZjMGFiXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
3718
saving picture
https://m.media-amazon.com/images/M/MV5BMTQ2NzQ2Njg0NF5BMl5BanBnXkFtZTYwNTIwNTU5._V1_UY268_CR0,0,182,268_AL_.jpg
3719
saving picture
https://m.media-amazon.com/images/M/MV5BNWNkOGYzOGYtZGE4OC00NDVjLWExZTYtZWE2YTA3MjI4YThlXkEyXkFqcGdeQXVyMTMxMTY0OTQ@._V1_UY268_CR8,0,182,268_AL_.jpg
3720
saving picture
https://m.media-amazon.com/images/M/MV5

saving picture
https://m.media-amazon.com/images/M/MV5BMjIxNzQzOTg4OV5BMl5BanBnXkFtZTcwMjM3MjM2Mg@@._V1_UX182_CR0,0,182,268_AL_.jpg
3762
saving picture
https://m.media-amazon.com/images/M/MV5BMWVhY2ZkMWQtNzkzMi00OWU0LWE5ZTAtZGQ2OTI2NDgwMWJlXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_UY268_CR4,0,182,268_AL_.jpg
3763
saving picture
https://m.media-amazon.com/images/M/MV5BYzBiY2JkYzUtODUwNS00YmRjLTg3ZDEtMzE2NGRjMzE3OGUzXkEyXkFqcGdeQXVyNjUwMzI2NzU@._V1_UY268_CR3,0,182,268_AL_.jpg
3764
saving picture
https://m.media-amazon.com/images/M/MV5BODg3MGMyNzgtYjI4Mi00Y2ZiLWIxYjQtNTY1ZWNjYzcwMmRkL2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNzc5MjA3OA@@._V1_UX182_CR0,0,182,268_AL_.jpg
3765
saving picture
https://m.media-amazon.com/images/M/MV5BNTM5Y2MxNDYtNzE0OC00YTI1LTg2ODQtZDRlMTk1NjMwYTg2XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
3766
saving picture
https://m.media-amazon.com/images/M/MV5BMzAyNDUwYzUtN2NlMC00ODliLWExMjgtMGMzNmYzZmUwYTg1XkEyXkFqcGdeQXVyNjc1NTYyMjg@._V1_UX182_CR0,0,182,268_AL_.jpg
3767
sa

saving picture
https://m.media-amazon.com/images/M/MV5BMTI5NDY1NDU1M15BMl5BanBnXkFtZTcwODcyNTMyMQ@@._V1_UY268_CR4,0,182,268_AL_.jpg
3796
saving picture
https://m.media-amazon.com/images/M/MV5BM2MzMTM5NzEtOGY2NC00OGMxLWFhNTctNWU2NTgyN2I2YmUzXkEyXkFqcGdeQXVyMTQ3Njg3MQ@@._V1_UY268_CR8,0,182,268_AL_.jpg
3797
saving picture
https://m.media-amazon.com/images/M/MV5BMTcxMzY1MTc1NV5BMl5BanBnXkFtZTcwOTY2OTQyMQ@@._V1_UY268_CR3,0,182,268_AL_.jpg
3798
saving picture
https://m.media-amazon.com/images/M/MV5BODk1ZWM4ZjItMjFhZi00MDMxLTgxNmYtODFhNWZlZTkwM2UwXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
3799
saving picture
https://m.media-amazon.com/images/M/MV5BYmU2NGU5MTUtYWRlNy00NzJiLWIxMTMtMDIxNDYxNjRlODg5XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
3800
saving picture
https://m.media-amazon.com/images/M/MV5BMTgxMWIwN2QtY2FkMi00YmM1LTg4NGEtMTQ4YTQwOTA2NWNkXkEyXkFqcGdeQXVyMTYzMTY1MjQ@._V1_UY268_CR2,0,182,268_AL_.jpg
3801
saving picture
https://m.media-amazon.com/images/M

saving picture
https://m.media-amazon.com/images/M/MV5BM2Y3Y2FlNTQtOTYzYS00NjdjLTlkNjEtNTE5NzRiMTZkNTgxXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UX182_CR0,0,182,268_AL_.jpg
3839
saving picture
https://m.media-amazon.com/images/M/MV5BODc2NDA5YTQtY2ViZi00OTk4LTlmMTgtYjM2YjljOTY5ZmYwXkEyXkFqcGdeQXVyNDE5MTU2MDE@._V1_UY268_CR1,0,182,268_AL_.jpg
3840
saving picture
https://m.media-amazon.com/images/M/MV5BNDVkYWMxNWEtNjc2MC00OGI5LWI3NmUtYWUwNDQyOTc3YmY5XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_UY268_CR0,0,182,268_AL_.jpg
3841
saving picture
https://m.media-amazon.com/images/M/MV5BMTQ5OTc0NDU1MF5BMl5BanBnXkFtZTYwNzk1OTI3._V1_UX182_CR0,0,182,268_AL_.jpg
3842
saving picture
https://m.media-amazon.com/images/M/MV5BN2Q2ZWU2ZGYtNDIzOC00YzRhLWI0ZTctYTI3M2EzNmNkOWNmXkEyXkFqcGdeQXVyNDYxNDE1NjQ@._V1_UX182_CR0,0,182,268_AL_.jpg
3843
saving picture
https://m.media-amazon.com/images/M/MV5BMTU5OTY5MjIzN15BMl5BanBnXkFtZTYwMTYwMzg5._V1_UX182_CR0,0,182,268_AL_.jpg
3844
saving picture
https://m.media-amazon.com/images/M/MV5BMjM